# 1. Librería

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
#from utils.Validator import *
import pickle
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 2. Importación de la data

In [2]:
train = pd.read_csv(r"../data/modeling/03_train_ppnn.csv",sep="|")   
test = pd.read_csv(r"../data/modeling/03_test_ppnn.csv",sep="|") 

## Train

In [3]:
train_sc2=train[(train.MTO_INGRESOS_MES>=150)&(train.MTO_INGRESOS_MES<=1e+6)]

In [4]:
train_sc3 = train[train['MTO_INGRESOS_MES']>1e+6]
train_sc3.shape

(63, 43)

In [5]:
train_sc3['n_cluster']=1

## Test

In [6]:
test_sc2=test[(test.MTO_INGRESOS_MES>=150)&(test.MTO_INGRESOS_MES<=1e+6)]

In [7]:
test_sc2.shape

(44068, 43)

In [8]:
test_sc3 = test[test['MTO_INGRESOS_MES']>1e+6]
test_sc3.shape

(9, 43)

In [9]:
test_sc3['n_cluster']=1

# 2. Exportación del modelo

In [10]:
model = pickle.load(open(r"../src/02_models/iforest_model_ppnn", 'rb'))

In [11]:
model.feature_names_in_

array(['MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS',
       'FLG_PERFIL_INGRESOS_3DS'], dtype=object)

# 3. Exportación de data

In [12]:
x_model=['MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS','FLG_PERFIL_INGRESOS_3DS']

In [13]:
vars_export=['PERIODO', 'CODCLAVECIC','TIPPER','MTO_INGRESOS_MES','FLG_PERFIL_INGRESOS_3DS','MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS','n_cluster']

In [14]:
# TRAIN
train_sc2['n_cluster']=model.predict(train_sc2[x_model])
#Alertas según el modelo Isolations 
alertas_train=train_sc2[train_sc2["n_cluster"]==-1]
alertas_train=pd.concat([alertas_train, train_sc3], ignore_index=True)
#Unimos las alertas del escalón 3 y 2
alertas_train[vars_export].to_csv(r"../data/modeling/04_predicted_train_ppnn.csv",index=False,sep="|")

In [15]:
# TEST
test_sc2['n_cluster']=model.predict(test_sc2[x_model])
#Alertas según el modelo Isolations 
alertas_test=test_sc2[test_sc2["n_cluster"]==-1]
alertas_test=pd.concat([alertas_test, test_sc3], ignore_index=True)
#Unimos las alertas del escalón 3 y 2
alertas_test[vars_export].to_csv(r"../data/modeling/04_predicted_test_ppnn.csv",index=False,sep="|")


In [16]:
alertas_test.shape # 16 modelo y 9 directo a evaluación

(25, 44)

In [17]:
alertas_test.n_cluster.value_counts()
# -1 modelo
# 1 ouliers (DaE)


-1    16
 1     9
Name: n_cluster, dtype: int64